In [1]:
import numpy as np
import tensorflow.keras
import pickle
from tensorflow.keras.layers import Dense,Conv2D,Activation,Dropout,ReLU,Flatten,MaxPooling2D,BatchNormalization
import cv2
import time
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import os
import kerastuner
%matplotlib inline

In [ ]:
# location of the data

Location='C:\path\to\your\PetImages'

# in this location i have two folders one is named as Cat and another one is Dog

# First we have to loop through both the folders and conver all the images into arrays 

def prepare_Data(Dir,image_size):
    data=[]                                                  # Creating empty list to store data
    
    categories=os.listdir(Dir)                               # Storing lables(folder names) in a list
    
    for category in categories:                              # Looping through each and every folder
        
        path=os.path.join(Dir,category)                      # joining current folder name with the path
        
        target=categories.index(category)                    # Storing the particular lable as target ['cat','Dog']
        
        for img in os.listdir(path):                         # Looping through the images 
            
            image=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE) # Reading the images
            try:
                image1=cv2.resize(image,(image_size,image_size)) # Images might be in different dimensions so resize it to same dimension
                
                data.append([image1,target])               # Appending resized image and its lable
            except Exception as e:
                pass
    np.random.shuffle(data)                                # Shuffling the data to avoid ["cat","cat","cat","dog","dog","gog"]
    X=[]
    y=[]
    for feature , lable in data:
        X.append(feature)                                  # Spliting X,y 
        y.append(lable)
    return X,y,categories
            

In [ ]:
# Let's Call the function to create dataset

X,y,data=image2array(Location,50)

In [ ]:
# Now our data is ready but it is a list we have to convert it into array
X=np.array(X)
X.shape

In [ ]:
# Now Shape of X is 3 dim but we need to convert it into 4 dim (-1,50,50,1) [No_sam,height,width,Channels]
X=np.array(X).reshape(-1,50,50,1)
y=np.array(y)

In [ ]:
# Each and every time we can not prepare data so it is safe to save our data somewhere

# for that i am going to use pickle

# Saving the data into pickle

import pickle

data_in=open('cat_dog_X.pickle','wb')
pickle.dump(X,data_in)
data_in.close()
data_y=open('cat_dog_y.pickle','wb')
pickle.dump(y,data_y)
data_y.close()

In [ ]:
# Now we can load our data from pickle

# To open data from pickle
x=open('cat_dog_X.pickle','rb')
X=pickle.load(x)
y=open('cat_dog_y.pickle','rb')
y=pickle.load(y)

In [ ]:
# Before feeding into model we have to normalize the data to get better performance

# for that we need to first spilt the data

X_train,X_test,y_train,y_test=train_test_split(X,y)

X_train=X_train/255

X_test=X_test/255


In [3]:
def one_hoty(y):
    z=np.zeros((len(y),len(np.unique(y))),dtype=int)
    for i in range(len(y)):
        z[i,y[i]]=1
    return z

In [ ]:
# if you want to use this classifier for multiclass then you need have one-hot encoded y

# y = one_hot(y)

In [ ]:
# Everuthing is ready now let's build the CNN model

model = Sequential()

# First Convolution layer
model.add(Conv2D(filters=65,kernel_size=4,input_shape=X.shape[1:],strides=1,padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=3))

# Second Convolution layer
model.add(Conv2D(filters=80,kernel_size=4,input_shape=X.shape[1:],strides=1,padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=3))

# Flatten
model.add(Flatten())

# Fully Connected layer1

model.add(Dense(290,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Dropout(0.3))

# Fully Connected layer12

model.add(Dense(70,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Dropout(0.3))

# Fully Connected layer3

model.add(Dense(170,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Dropout(0.3))

# Fully Connected layer4

model.add(Dense(50,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Dropout(0.3))

# Fully Connected layer5

model.add(Dense(130,kernel_initializer='he_normal'))
model.add(Activation('relu'))
# model.add(Dropout(0.3))

# Output Layer

model.add(Dense(1))
model.add(Activation('sigmoid'))

# Compling the model

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [2]:
# This is a binaer Classifier if you want to use it for multiclass classification then use softmax in output layer a
# Also use one hot encoder to convert y.

In [ ]:
model.fit(X_train, y_train, batch_size=100, epochs=100, validation_data=(X_test, y_test))

In [ ]:
# after running this model for 100 epoch on Gogle colab i got 88 percent accuracy on test data 
# if we tune even more we may get even better results

# Keras Tuner

In [4]:
# Our Model works well but not giving good accuracy. for that we have to tune our model
# I am going to use Keras Tuner

from kerastuner.tuners import RandomSearch

def build_function(hp):  # hp is the hyperparameter
    
    # first layer will be unique because it consist input shape so we are defining it outside the loop'
    
    model = Sequential()
    
    model.add(Conv2D(filters=hp.Int('inupt_layer',10,100,5),kernel_size=4,input_shape=X_train.shape[1:],strides=1,
                     padding='same',kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size =4))
    
    # Now it's time to decide how many con & Dense layer we need for good performance and 
    
    # for that we are going to use just a for loop
    
    for i in range(hp.Int("conv_layers",1,3)):
        
        # Here we need to specify the convolution layer without input size
        # It will randomly pic no of layer from 1 to 5
        
        model.add(Conv2D(filters=hp.Int(f'hidden_conv_neurons{i}_',10,100,5),kernel_size=3,
              strides=1,padding='same',kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size =4))
    
    # after conolution layer we need to flatten the data so let's add one Flatten layer
    
    model.add(Flatten())  # note it is outside the loop
    
    # here we have to use one more for loop for Dense layer
    for j in range(hp.Int('Dense_layer',1,10)):
        # It will randomly pic no of layer from 1 to 10
        model.add(Dense(units=hp.Int(f"Dense_{j}_layer",50,300,20)))
        
        model.add(Dropout(0.2))             # To avaoid overfitting we are adding dropout it is a regularization technique
        
        model.add(Activation('relu'))
    
    # Finally we need to add output layer
    
    model.add(Dense(1))
    model.add(Activation('sigmoid')) # This is a binary classification so we are using sigmoid
    
    # Now we need to compile the model
    
    model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
LOG_DIR = f"{int(time.time())}"

tuner = RandomSearch(
    build_function,
    objective='val_accuracy',
    max_trials=5,              
    executions_per_trial=3,    
    directory=LOG_DIR)


In [ ]:
tuner.search(X_train,y_train,verbose=2,
             epochs=20,
             batch_size=35,
             validation_data=(X_test, y_test))